In [1]:
import numpy as np
import glob
import requests

In [2]:
PATH = '../lab1/ustawy/'
file_names = sorted(glob.glob(PATH + "*"))
file_names[:5]

['../lab1/ustawy/1993_599.txt',
 '../lab1/ustawy/1993_602.txt',
 '../lab1/ustawy/1993_645.txt',
 '../lab1/ustawy/1993_646.txt',
 '../lab1/ustawy/1994_150.txt']

In [3]:
law_acts =[]
for file_name in file_names:
    with open(file_name, "r", encoding="utf-8") as f:
        law_acts.append(f.read())

# Zadanie 2
Znalezienie tokenów z korpusu w postaci tupli (lemat (downcased), kategoria morfosyntaktyczna)

In [4]:
%%time
tokens_per_act = []

for law_act in law_acts:
    tokens = []
    response_lines = requests.post('http://localhost:9200', law_act.encode('utf-8')).content.decode('utf-8').split('\n')
    for line in response_lines:
        line_words = line.split('\t')
        if line_words[0] == '' and len(line_words) >= 2:
            tokens.append((line_words[1].lower().strip(), line_words[2].split(':')[0]))
    tokens_per_act.append(tokens)

CPU times: user 21 s, sys: 1.62 s, total: 22.6 s
Wall time: 1h 31min 49s


In [5]:
tokens_per_act[0][:10]

[('dziennik', 'brev'),
 ('.', 'interp'),
 ('ustawa', 'brev'),
 ('.', 'interp'),
 ('z', 'prep'),
 ('1993', 'adj'),
 ('rok', 'brev'),
 ('.', 'interp'),
 ('numer', 'brev'),
 ('129', 'num')]

# Zadanie 3
Obliczenie licznika bigramów powstałych z opisanych wyżej tokenów

In [6]:
%%time
bigrams_counter = {}
for tokens in tokens_per_act:
    bigrams = zip(tokens[:-1], tokens[1:])
    for bigram in bigrams:
        bigrams_counter[bigram] = bigrams_counter.get(bigram, 0) + 1

CPU times: user 5.67 s, sys: 19.4 ms, total: 5.69 s
Wall time: 5.69 s


# Zadanie 4
Odfiltrowanie bigramów zawierających nie litery

In [7]:
%%time
new_bigrams_counter = {}
for bigram, count in bigrams_counter.items():
    if bigram[0][0].isalpha() and bigram[1][0].isalpha():
        new_bigrams_counter[bigram] = count
bigrams_counter = new_bigrams_counter

CPU times: user 355 ms, sys: 16.1 ms, total: 371 ms
Wall time: 368 ms


# Zadanie 6
Obliczenie LLR dla znalezionych bigramów

In [8]:
total_number_of_bigrams = sum(list(bigrams_counter.values()))
total_number_of_bigrams

2768350

In [9]:
# compute counter of bigrams starting with a
bigrams_starting_with_a_counter = {}
for bigram, count in bigrams_counter.items():
    a = bigram[0]
    bigrams_starting_with_a_counter[a] = bigrams_starting_with_a_counter.get(a, 0) + count
    
# compute counter of bigrams ending with b
bigrams_ending_with_b_counter = {}
for bigram, count in bigrams_counter.items():
    b = bigram[1]
    bigrams_ending_with_b_counter[b] = bigrams_ending_with_b_counter.get(b, 0) + count

In [10]:
def H(k):
    N = np.sum(k)
    return np.sum(k/N * np.log(k/N + (k==0)))

In [11]:
%%time
bigrams_llr = {}
for bigram, count in bigrams_counter.items():
    a, b = bigram
    a_and_b = count
    a_without_b = bigrams_starting_with_a_counter[a] - a_and_b
    b_but_not_a = bigrams_ending_with_b_counter[b] - a_and_b
    neither_a_nor_b = total_number_of_bigrams - bigrams_starting_with_a_counter[a] \
        - bigrams_ending_with_b_counter[b] + a_and_b
    
    k = np.array([[a_and_b, b_but_not_a], [a_without_b, neither_a_nor_b]])
    row_sums = np.array([a_and_b + b_but_not_a, a_without_b + neither_a_nor_b])
    col_sums = np.array([a_and_b + a_without_b, b_but_not_a + neither_a_nor_b])
    llr = 2 * np.sum(k) * (H(k) - H(row_sums) - H(col_sums))
    bigrams_llr[bigram] = llr

CPU times: user 46.8 s, sys: 523 ms, total: 47.3 s
Wall time: 47.3 s


In [12]:
list(bigrams_llr.items())[:10]

[((('ustawa', 'subst'), ('z', 'prep')), 42102.727935503),
 ((('z', 'prep'), ('dzień', 'subst')), 53719.90512407017),
 ((('o', 'prep'), ('zmiana', 'subst')), 4524.518585883802),
 ((('zmiana', 'subst'), ('ustawa', 'subst')), 4469.472597983152),
 ((('ustawa', 'subst'), ('o', 'prep')), 4768.254868144018),
 ((('o', 'prep'), ('podatek', 'subst')), 1650.9791429647435),
 ((('podatek', 'subst'), ('od', 'prep')), 2545.2504767769183),
 ((('od', 'prep'), ('towar', 'subst')), 933.3632228124396),
 ((('towar', 'subst'), ('i', 'conj')), 1675.4625112097674),
 ((('i', 'conj'), ('usługa', 'subst')), 1884.330998092189)]

# Zadanie 7
Wyznaczenie partycji bigramów zawierających tokeny z tymi samymi kategoriami

In [13]:
partitions = {}
partitions_size = {}

for bigram, count in bigrams_counter.items():
    partition = (bigram[0][1], bigram[1][1])
    partitions[partition] = partitions.get(partition, []) + [(bigram[0][0], bigram[1][0])]
    partitions_size[partition] = partitions_size.get(partition, 0) + count

# Zadanie 8
Wyznaczenie 10 najliczniejszych partycji

In [14]:
largest_partitions = sorted(list(partitions_size.items()), key=(lambda x: (-x[1], x[0])))[:10]
largest_partitions

[(('prep', 'subst'), 327841),
 (('subst', 'subst'), 293793),
 (('subst', 'adj'), 274962),
 (('adj', 'subst'), 188439),
 (('subst', 'prep'), 173782),
 (('subst', 'conj'), 85165),
 (('conj', 'subst'), 84274),
 (('prep', 'adj'), 79485),
 (('ger', 'subst'), 76538),
 (('prep', 'brev'), 67137)]

# Zadanie 9
Wyznaczenie 5 bigramów z najwyższym LLR reprezentujących każdą z partycji

In [15]:
categories_representatives = {}
sorted_bigrams = sorted(list(bigrams_llr.items()), key=(lambda x: (-x[1], x[0])))
for partition, size in largest_partitions:
    representatives_found = 0
    for bigram, llr in sorted_bigrams:
        if bigram[0][1] == partition[0] and bigram[1][1] == partition[1]:
            categories_representatives[partition] = categories_representatives.get(partition, []) + [((bigram[0][0], bigram[1][0]), llr)]
            representatives_found += 1
            if representatives_found == 5:
                break

In [16]:
for partition, representatives in categories_representatives.items():
    print('Partition:', partition)
    print('Representatives:')
    for r in representatives:
        print(r)
    print()

Partition: ('prep', 'subst')
Representatives:
(('z', 'dzień'), 53719.90512407017)
(('na', 'podstawa'), 47389.81442574456)
(('do', 'sprawa'), 46331.317170673115)
(('w', 'droga'), 32059.638214459912)
(('od', 'dzień'), 31767.932084897366)

Partition: ('subst', 'subst')
Representatives:
(('droga', 'rozporządzenie'), 53978.44612045842)
(('skarb', 'państwo'), 21933.48773133111)
(('rada', 'minister'), 18342.06970079752)
(('terytorium', 'rzeczpospolita'), 14155.022492691465)
(('ochrona', 'środowisko'), 14029.946469116026)

Partition: ('subst', 'adj')
Representatives:
(('minister', 'właściwy'), 71029.07051601577)
(('rzeczpospolita', 'polski'), 41742.7716811986)
(('jednostka', 'organizacyjny'), 24609.374292130935)
(('samorząd', 'terytorialny'), 23394.1293598482)
(('produkt', 'leczniczy'), 21913.360666442946)

Partition: ('adj', 'subst')
Representatives:
(('który', 'mowa'), 249005.7458068642)
(('niniejszy', 'ustawa'), 21509.338101544676)
(('następujący', 'zmiana'), 18174.091280081757)
(('odrębny'

# Wnioski

1. What types of bigrams have been found?

(Poniżej będę używał nazw części mowy, a nie tagów morfoskładniowych, które im odpowiadają)
Dla najliczniejszej partycji bigramów (przyimek+rzeczownik) znaleziono popularne kolokacje np. 'na podstawie', 'w drodze/gę' występujące często w języku normalnie. Dwa rzeczowniki to popularne dwuczłonowe nazwy np. 'rada ministrów', 'skarb państwa'. Analogicznie dwuczłonowe nazwy są też reprezentowane przez rzeczownik+przymiotnik np. 'Rzeczpospolita Polska', 'samorząd terytorialny. Przymiotnik+rzeczownik to często występujące w korpusie połączenia np. 'następująca zmiana', 'walne zgromadzenie', ale nie są to typowe konstrukcje językowe jak przyimek+rzeczownik. Rzeczownik+przyimek bardzo podobnie jak przymiotnik+rzeczownik - bigramy częste w korpusie, nie tak częste ogólnie np. 'mowa w', 'ustawa z'. Bigramy składające się z rzeczownika+spójnika większości nie wydają się reprezentować sobą nic ciekawego, jedynie 'imię i' prawdopodobnie pochodzi z popularnej frazy 'imię i nazwisko'. Podobnie bigramy spójnik+rzeczownik nie wydają się mieć głębszego znaczenia (oprócz 'i nazwisko' podobnie jak 'imię i'). Przyimek+przymiotnik to ta sama kategoria jak przymiotnik+rzeczownik - popularne w korpusie, występujące w języku, ale nie są to typowe konstrukcje językowe, np. 'o którym', 'za każdym'. Rzeczownik odsłowny+rzeczownik to popularne połączenia słów, które nie występują super często (przynajmniej jedno ze słów), ale jak już wystąpią to zazwyczaj są częścią bigramu np. 'zasięgnąć opinii', 'pozbawić wolności'. Ostatnia kategoria składa się cała z określeń miejsca/części artykułu/czasu, np 'w artykule', 'w punkcie'.

2. Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?

Większość partycji jest reprezentowana przez wartościowe bigramy. Na szczególną uwagę zasługują na pewno:

- przyimek+rzeczownik - są najpopularniejsze i najbardziej przekładają się na cały język

- rzeczownik+rzeczownik, rzeczownik+przymiotnik, rzeczownik odsłowny+rzeczownik - reprezentują głównie dwuczłonowe nazwy, albo bigramy których słowa (albo przynajmniej 1 z nich) rzadko występują osobno

- przymiotnik+rzeczownik, rzeczownik+przyimek i przyimek+przymiotnik - reprezentują bigramy popularne w korpusie i specyficzne dla niego (nie aż tak popularne ogólnie)

3. Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?

Samo zastosowanie LLR zwróci nam dobre bigramy z punktu widzenia częstości ich występowania (tutaj np. 'ustawa z', 'z dnia'). Nie są one w żaden sposób opisane, czy zaklasyfikowane. Użycie kategorii syntaktycznych po pierwsze pozwala łatwiej zrozumieć co reprezentują sobą bigramy (np. przez analizę partycji), po drugie umożliwia odfiltrowanie często występujących typowych językowych struktów i skupienie się np. na dwuczłonowych nazwach, czy typowych multiwordach dla korpusu. Obie metody mają swoje zastosowanie, ale dla przeciętnego użytkownika prawdopodobnie najlepsze będzie zastosowanie połączenia obu (jak w zadaniu 9)

4. Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?

Tagowanie morfoskładniowe mogłoby być użyte np. w procesie automatycznego tłumaczenia tekstu. Typowe struktury zdań mogłyby być opisywane przez gramatykę składającą się z symboli reprezentujących tagi morfoskładniowe. Innym potencjalnym zastosowaniem mogłaby być autokorekta błędnych końcówek słów, w języku polskim bardzo łatwo przekręcając końcówkę zmienić część mowy, np. Polska i polski.
